![NeuronUnit Logo](https://raw.githubusercontent.com/scidash/assets/master/logos/neuronunit-logo-text.png)
# Chapter 3
Back to [Chapter 2](chapter2.ipynb)

In the second chapter we tested a real ion channel model using data from the [OpenWorm](http://openworm.org) project.  Here we'll test a reduced neuron model using data from the [NeuroElectro](http://neuroelectro.org) project and from the [Allen Brain Institute Cell Types](http://celltypes.brain-map.org) database.  
### We'll test an Izhikevich model against data from a Layer V pyramidal cell in primary visual cortex

In [1]:
%matplotlib inline
import os,sys
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
import sciunit
import neuronunit
from neuronunit import aibs
from neuronunit.models.reduced import ReducedModel

/Users/rgerkin/Dropbox/miniconda3/lib/python3.5/site-packages/pyNN/neuron/__init__.py:14: UserWarning: mpi4py not available
  warnings.warn("mpi4py not available")


In [2]:
import quantities as pq
from neuronunit import tests as nu_tests, neuroelectro
neuron = {'nlex_id': 'nifext_50'} # Layer V pyramidal cell
tests = []

dataset_id = 354190013  # Internal ID that AIBS uses for a particular Scnn1a-Tg2-Cre 
                        # Primary visual area, layer 5 neuron.
    
# Obtain the empirical rheobase current from the Allen Brain Insitute Cell Types database.  
observation = aibs.get_observation(dataset_id,'rheobase')
rheobase_test = nu_tests.RheobaseTest(observation=observation)
tests += [rheobase_test]
    
test_class_params = [(nu_tests.InputResistanceTest,None),
                     (nu_tests.TimeConstantTest,None),
                     (nu_tests.CapacitanceTest,None),
                     (nu_tests.RestingPotentialTest,None),
                     (nu_tests.InjectedCurrentAPWidthTest,None),
                     (nu_tests.InjectedCurrentAPAmplitudeTest,None),
                     (nu_tests.InjectedCurrentAPThresholdTest,None)
                    ]

# Obtain all other parameters from neuroelectro.org.
for cls,params in test_class_params:
    observation = cls.neuroelectro_summary_observation(neuron)
    tests += [cls(observation,params=params)]
    
# A hook to update all tests after the RheobaseTest to use the rheobase current (to produce exactly one AP)
def update_amplitude(test,tests,score):
    rheobase = score.prediction['value']
    for test in tests[1:]:
        if 'Injected' in test.name:
            # Set current injection to just suprathreshold
            test.params['injected_square_current']['amplitude'] = rheobase*1.01 
    
hooks = {tests[0]:{'f':update_amplitude}}
suite = sciunit.TestSuite(tests,name="vm_suite",hooks=hooks)

Getting Rheobase cached data value for from AIBS dataset 354190013
Getting Input Resistance data values from neuroelectro.org
{'nlex': 'nifext_50', 'e__name': 'Input Resistance', 'e': None, 'n__name': None, 'n': None}
http://www.neuroelectro.org/api/1/nes/?nlex=nifext_50&e__name=Input+Resistance
Getting Membrane Time Constant data values from neuroelectro.org
{'nlex': 'nifext_50', 'e__name': 'Membrane Time Constant', 'e': None, 'n__name': None, 'n': None}
http://www.neuroelectro.org/api/1/nes/?nlex=nifext_50&e__name=Membrane+Time+Constant
Getting Cell Capacitance data values from neuroelectro.org
{'nlex': 'nifext_50', 'e__name': 'Cell Capacitance', 'e': None, 'n__name': None, 'n': None}
http://www.neuroelectro.org/api/1/nes/?nlex=nifext_50&e__name=Cell+Capacitance
Getting Resting membrane potential data values from neuroelectro.org
{'nlex': 'nifext_50', 'e__name': 'Resting membrane potential', 'e': None, 'n__name': None, 'n': None}
http://www.neuroelectro.org/api/1/nes/?nlex=nifext_50&

### The Izhikevich model here is defined in an NeuroML2/LEMS file.  
### We will run it using the jNeuroML backend for simplicity, although this is *much* slower than the native NEURON backend.  

In [3]:
# This example is from https://github.com/OpenSourceBrain/IzhikevichModel.
for rel_path in ['.','docs','../docs']:
    # Check to see if this is the neuronunit/docs directory.
    DOCS_PATH = os.path.abspath(os.path.join(os.getcwd(),rel_path)) 
    if DOCS_PATH.endswith('neuronunit/docs'):
        break
assert DOCS_PATH.endswith('neuronunit/docs'), ("Could not find the path to neuronunit/docs. "
                                               "Change the DOCS_PATH variable to the path to "
                                               "your neuronunit/docs directory.")
LEMS_MODEL_PATH = os.path.join(DOCS_PATH,'data/NeuroML2/LEMS_2007One.xml')
model = ReducedModel(LEMS_MODEL_PATH,name='vanilla',backend='jNeuroML')

### Run a series of tests.  The RheobaseTest is run first to obtain the rheobase current to use in all the other tests.

In [4]:
score_matrix = suite.judge(model)

Injected 300.0 current and got 17 spikes
Injected -0.0 current and got 0 spikes
Injected 150.0 current and got 8 spikes
Injected 75.0 current and got 2 spikes
Injected 37.5 current and got 0 spikes
Injected 56.25 current and got 1 spikes
Injected 46.875 current and got 0 spikes
Injected 51.5625 current and got 0 spikes
Injected 53.90625 current and got 0 spikes
Injected 55.078125 current and got 1 spikes
Injected 54.4921875 current and got 0 spikes
Highest subthreshold current is 54.49 pA
Lowest suprathreshold current is 55.08 pA


### Now we can sweep across a parameter (the resting potential) and run the same test suite on a model corresponding to each value of this parameter.

In [ ]:
models = []
for vr in np.linspace(-80,-50,4):
    model = ReducedModel(LEMS_MODEL_PATH, 
                         name='V_rest=%dmV' % vr, 
                         attrs={'//izhikevich2007Cell':
                                    {'vr':'%d mV' % vr}
                               })
    #model.skip_run = True
    models.append(model)
suite.set_verbose(False) # Turn off most print statements.  
score_matrix = suite.judge(models)

In [ ]:
score_matrix.show_mean = True # Show the mean value across test scores. 
                              # The mean is computed using the sort_key attribute, which is in the range [0,1] for
                              # all Score types
score_matrix.sortable = True # Make the ScoreMatrix sortable (and searchable)
score_matrix

### Let's take a look at a sweep from one of these models ($V_{rest} = -55 mV$) obtained at the rheobase current.  

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 18
score_matrix['RheobaseTest']['V_rest=-60mV'].plot_vm() # Plot the rheobase current from the model with V_rest = -60 mV